In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

vncorenlp_dir = '/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2'

# Kiểm tra sự tồn tại của các tệp
required_files = [
    'VnCoreNLP-1.2.jar',
    'models/wordsegmenter/vi-vocab',
    'models/wordsegmenter/wordsegmenter.rdr'
]

for file in required_files:
    file_path = os.path.join(vncorenlp_dir, file)  # Kết hợp đúng cách
    print(f'Đường dẫn kiểm tra: {file_path}')  # In ra đường dẫn để kiểm tra
    if os.path.exists(file_path):
        print(f'{file} đã có mặt tại {file_path}')
    else:
        print(f'{file} không tìm thấy tại {file_path}')


Đường dẫn kiểm tra: /content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2/VnCoreNLP-1.2.jar
VnCoreNLP-1.2.jar đã có mặt tại /content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2/VnCoreNLP-1.2.jar
Đường dẫn kiểm tra: /content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2/models/wordsegmenter/vi-vocab
models/wordsegmenter/vi-vocab đã có mặt tại /content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2/models/wordsegmenter/vi-vocab
Đường dẫn kiểm tra: /content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2/models/wordsegmenter/wordsegmenter.rdr
models/wordsegmenter/wordsegmenter.rdr đã có mặt tại /content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.2/VnCoreNLP-1.2/models/wordsegmenter/wordsegmenter.rdr


In [ ]:
!pip install emoji
!pip install requests
!pip install regex
!pip install vncorenlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=eb35c69634a59a50928885b5740aeafcd0c53c5379d18b4c16b2726e5524e2eb
  Stored in directory: /root/.cache/pip/wheels/80/ad/d4/9e1a0939f63331a3898f2a951a368bbf0d69f7b027cae4d66b
Successfully built vncorenlp


In [ ]:
# https://github.com/behitek/text-classification-tutorial/blob/master/text_classification_tutorial.ipynb
import os
import emoji
import urllib
import requests
import regex as re

from io import StringIO
from vncorenlp import VnCoreNLP
from transformers import pipeline


class VietnameseTextCleaner: # https://ihateregex.io
    VN_CHARS = 'áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ'

    @staticmethod
    def remove_html(text):
        return re.sub(r'<[^>]*>', '', text)

    @staticmethod
    def remove_emoji(text):
        return emoji.replace_emoji(text, '')

    @staticmethod
    def remove_url(text):
        return re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', '', text)

    @staticmethod
    def remove_email(text):
        return re.sub(r'[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+', '', text)

    @staticmethod
    def remove_phone_number(text):
        return re.sub(r'^[\+]?[(]?[0-9]{3}[)]?[-\s\.]?[0-9]{3}[-\s\.]?[0-9]{4,6}$', '', text)

    @staticmethod
    def remove_hashtags(text):
        return re.sub(r'#\w+', '', text)

    @staticmethod
    def remove_unnecessary_characters(text):
        text = re.sub(fr"[^\sa-zA-Z0-9{VietnameseTextCleaner.VN_CHARS}]", ' ', text)
        return re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace

    @staticmethod
    def process_text(text):
        text = VietnameseTextCleaner.remove_html(text)
        text = VietnameseTextCleaner.remove_emoji(text)
        text = VietnameseTextCleaner.remove_url(text)
        text = VietnameseTextCleaner.remove_email(text)
        text = VietnameseTextCleaner.remove_phone_number(text)
        text = VietnameseTextCleaner.remove_hashtags(text)
        return VietnameseTextCleaner.remove_unnecessary_characters(text)


class VietnameseToneNormalizer:
    VOWELS_TABLE = [
        ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
        ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
        ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
        ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
        ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
        ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
        ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
        ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
        ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
        ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
        ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
        ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']
    ]

    # VOWELS_TO_IDS = {}
    # for i, row in enumerate(VOWELS_TABLE):
    #     for j, char in enumerate(row[:-1]):
    #         VOWELS_TO_IDS[char] = (i, j)
    VOWELS_TO_IDS = {
        'a': (0, 0), 'à': (0, 1), 'á': (0, 2), 'ả': (0, 3), 'ã': (0, 4), 'ạ': (0, 5),
        'ă': (1, 0), 'ằ': (1, 1), 'ắ': (1, 2), 'ẳ': (1, 3), 'ẵ': (1, 4), 'ặ': (1, 5),
        'â': (2, 0), 'ầ': (2, 1), 'ấ': (2, 2), 'ẩ': (2, 3), 'ẫ': (2, 4), 'ậ': (2, 5),
        'e': (3, 0), 'è': (3, 1), 'é': (3, 2), 'ẻ': (3, 3), 'ẽ': (3, 4), 'ẹ': (3, 5),
        'ê': (4, 0), 'ề': (4, 1), 'ế': (4, 2), 'ể': (4, 3), 'ễ': (4, 4), 'ệ': (4, 5),
        'i': (5, 0), 'ì': (5, 1), 'í': (5, 2), 'ỉ': (5, 3), 'ĩ': (5, 4), 'ị': (5, 5),
        'o': (6, 0), 'ò': (6, 1), 'ó': (6, 2), 'ỏ': (6, 3), 'õ': (6, 4), 'ọ': (6, 5),
        'ô': (7, 0), 'ồ': (7, 1), 'ố': (7, 2), 'ổ': (7, 3), 'ỗ': (7, 4), 'ộ': (7, 5),
        'ơ': (8, 0), 'ờ': (8, 1), 'ớ': (8, 2), 'ở': (8, 3), 'ỡ': (8, 4), 'ợ': (8, 5),
        'u': (9, 0), 'ù': (9, 1), 'ú': (9, 2), 'ủ': (9, 3), 'ũ': (9, 4), 'ụ': (9, 5),
        'ư': (10, 0), 'ừ': (10, 1), 'ứ': (10, 2), 'ử': (10, 3), 'ữ': (10, 4), 'ự': (10, 5),
        'y': (11, 0), 'ỳ': (11, 1), 'ý': (11, 2), 'ỷ': (11, 3), 'ỹ': (11, 4), 'ỵ': (11, 5)
    }

    VINAI_NORMALIZED_TONE = {
        'òa': 'oà', 'Òa': 'Oà', 'ÒA': 'OÀ',
        'óa': 'oá', 'Óa': 'Oá', 'ÓA': 'OÁ',
        'ỏa': 'oả', 'Ỏa': 'Oả', 'ỎA': 'OẢ',
        'õa': 'oã', 'Õa': 'Oã', 'ÕA': 'OÃ',
        'ọa': 'oạ', 'Ọa': 'Oạ', 'ỌA': 'OẠ',
        'òe': 'oè', 'Òe': 'Oè', 'ÒE': 'OÈ',
        'óe': 'oé', 'Óe': 'Oé', 'ÓE': 'OÉ',
        'ỏe': 'oẻ', 'Ỏe': 'Oẻ', 'ỎE': 'OẺ',
        'õe': 'oẽ', 'Õe': 'Oẽ', 'ÕE': 'OẼ',
        'ọe': 'oẹ', 'Ọe': 'Oẹ', 'ỌE': 'OẸ',
        'ùy': 'uỳ', 'Ùy': 'Uỳ', 'ÙY': 'UỲ',
        'úy': 'uý', 'Úy': 'Uý', 'ÚY': 'UÝ',
        'ủy': 'uỷ', 'Ủy': 'Uỷ', 'ỦY': 'UỶ',
        'ũy': 'uỹ', 'Ũy': 'Uỹ', 'ŨY': 'UỸ',
        'ụy': 'uỵ', 'Ụy': 'Uỵ', 'ỤY': 'UỴ',
    }


    @staticmethod
    def normalize_unicode(text):
        char1252 = r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
        charutf8 = r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
        char_map = dict(zip(char1252.split('|'), charutf8.split('|')))
        return re.sub(char1252, lambda x: char_map[x.group()], text.strip())


    @staticmethod
    def normalize_sentence_typing(text, vinai_normalization=False):
        # https://github.com/VinAIResearch/BARTpho/blob/main/VietnameseToneNormalization.md
        if vinai_normalization: # Just simply replace the wrong tone with the correct one defined by VinAI
            for wrong, correct in VietnameseToneNormalizer.VINAI_NORMALIZED_TONE.items():
                text = text.replace(wrong, correct)
            return text.strip()

        # Or you can use this algorithm developed by Behitek to normalize Vietnamese typing in a sentence
        words = text.strip().split()
        for index, word in enumerate(words):
            cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
            if len(cw) == 3: cw[1] = VietnameseToneNormalizer.normalize_word_typing(cw[1])
            words[index] = ''.join(cw)
        return ' '.join(words)


    @staticmethod
    def normalize_word_typing(word):
        if not VietnameseToneNormalizer.is_valid_vietnamese_word(word): return word
        chars, vowel_indexes = list(word), []
        qu_or_gi, tonal_mark = False, 0

        for index, char in enumerate(chars):
            if char not in VietnameseToneNormalizer.VOWELS_TO_IDS: continue
            row, col = VietnameseToneNormalizer.VOWELS_TO_IDS[char]
            if index > 0 and (row, chars[index - 1]) in [(9, 'q'), (5, 'g')]:
                chars[index] = VietnameseToneNormalizer.VOWELS_TABLE[row][0]
                qu_or_gi = True

            if not qu_or_gi or index != 1: vowel_indexes.append(index)
            if col != 0:
                tonal_mark = col
                chars[index] = VietnameseToneNormalizer.VOWELS_TABLE[row][0]

        if len(vowel_indexes) < 2:
            if qu_or_gi:
                index = 1 if len(chars) == 2 else 2
                if chars[index] in VietnameseToneNormalizer.VOWELS_TO_IDS:
                    row, _ = VietnameseToneNormalizer.VOWELS_TO_IDS[chars[index]]
                    chars[index] = VietnameseToneNormalizer.VOWELS_TABLE[row][tonal_mark]
                else: chars[1] = VietnameseToneNormalizer.VOWELS_TABLE[5 if chars[1] == 'i' else 9][tonal_mark]
                return ''.join(chars)
            return word

        for index in vowel_indexes:
            row, _ = VietnameseToneNormalizer.VOWELS_TO_IDS[chars[index]]
            if row in [4, 8]: # ê, ơ
                chars[index] = VietnameseToneNormalizer.VOWELS_TABLE[row][tonal_mark]
                return ''.join(chars)

        index = vowel_indexes[0 if len(vowel_indexes) == 2 and vowel_indexes[-1] == len(chars) - 1 else 1]
        row, _ = VietnameseToneNormalizer.VOWELS_TO_IDS[chars[index]]
        chars[index] = VietnameseToneNormalizer.VOWELS_TABLE[row][tonal_mark]
        return ''.join(chars)


    @staticmethod
    def is_valid_vietnamese_word(word):
        vowel_indexes = -1
        for index, char in enumerate(word):
            if char not in VietnameseToneNormalizer.VOWELS_TO_IDS: continue
            if vowel_indexes in [-1, index - 1]: vowel_indexes = index
            else: return False
        return True


class VietnameseTextPreprocessor:
    def __init__(self, vncorenlp_dir='./VnCoreNLP', extra_teencodes=None, max_correction_length=512):
        self.vncorenlp_dir = vncorenlp_dir
        self.extra_teencodes = extra_teencodes
        self._load_vncorenlp()
        self._build_teencodes()

        self.max_correction_length = max_correction_length
        self.corrector = pipeline(
            'text2text-generation', model='bmd1905/vietnamese-correction-v2',
            torch_dtype='bfloat16', device_map='auto', num_workers=os.cpu_count()
        )
        print('bmd1905/vietnamese-correction-v2 is loaded successfully.')


    def _load_vncorenlp(self):
        self.word_segmenter = None
        if self._get_vncorenlp_files('/VnCoreNLP-1.2.jar') and \
           self._get_vncorenlp_files('/models/wordsegmenter/vi-vocab') and \
           self._get_vncorenlp_files('/models/wordsegmenter/wordsegmenter.rdr'):
            self.word_segmenter = VnCoreNLP(self.vncorenlp_dir + '/VnCoreNLP-1.2.jar', annotators='wseg', quiet=False)
            print('VnCoreNLP word segmenter is loaded successfully.')
        else: print('Failed to load VnCoreNLP word segmenter.')


    def _get_vncorenlp_files(self, url_slash):
        local_path = self.vncorenlp_dir + url_slash
        if os.path.exists(local_path): return True

        # Check if the folder contains the local_path exists, if not, create it.
        if not os.path.exists(os.path.dirname(local_path)):
            os.makedirs(os.path.dirname(local_path))

        download_url = 'https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master' + url_slash
        try:
            print(f'Downloading {download_url} to {local_path}')
            return urllib.request.urlretrieve(download_url, local_path)
        except urllib.error.HTTPError as e:
            print(f'Failed to download {download_url} due to {e}')
            return False


    def _build_teencodes(self):
        self.teencodes = {
            'ok': ['okie', 'okey', 'ôkê', 'oki', 'oke', 'okay', 'okê'],
            'không': ['kg', 'not', 'k', 'kh', 'kô', 'hok', 'ko', 'khong'], 'không phải': ['kp'],
            'cảm ơn': ['tks', 'thks', 'thanks', 'ths', 'thank'], 'hồi đó': ['hùi đó'], 'muốn': ['mún'],

            'rất tốt': ['perfect', '❤️', '😍'], 'dễ thương': ['cute'], 'yêu': ['iu'], 'thích': ['thik'],
            'tốt': [
                'gud', 'good', 'gút', 'tot', 'nice',
                'hehe', 'hihi', 'haha', 'hjhj', 'thick', '^_^', ':)', '=)'
                '👍', '🎉', '😀', '😂', '🤗', '😙', '🙂'
            ],
            'bình thường': ['bt', 'bthg'], 'hàg': ['hàng'],
            'không tốt':  ['lol', 'cc', 'huhu', ':(', '😔', '😓'],
            'tệ': ['sad', 'por', 'poor', 'bad'], 'giả mạo': ['fake'],

            'quá': ['wa', 'wá', 'qá'], 'được': ['đx', 'dk', 'dc', 'đk', 'đc'],
            'với': ['vs'], 'gì': ['j'], 'rồi': ['r'], 'mình': ['m', 'mik'],
            'thời gian': ['time'], 'giờ': ['h'],
        }
        if self.extra_teencodes:
            for key, values in self.extra_teencodes.items():
                if any(len(value.split()) > 1 for value in values):
                    raise ValueError('The values for each key in extra_teencodes must be single words.')
                self.teencodes.setdefault(key, []).extend(values)

        self.teencodes = {word: key for key, values in self.teencodes.items() for word in values}
        teencode_url = 'https://gist.githubusercontent.com/behitek/7d9441c10b3c2739499fc5a4d9ea06fb/raw/df939245b3e841b62af115be4dcb3516dadc9fc5/teencode.txt'
        response = requests.get(teencode_url)

        if response.status_code == 200:
            text_data = StringIO(response.text)
            for pair in text_data:
                teencode, true_text = pair.split('\t')
                self.teencodes[teencode.strip()] = true_text.strip()
            self.teencodes = {k: self.teencodes[k] for k in sorted(self.teencodes)}
        else: print('Failed to fetch teencode.txt from', teencode_url)


    def normalize_teencodes(self, text):
        words = []
        for word in text.split():
            words.append(self.teencodes.get(word, word))
        return ' '.join(words)


    def correct_vietnamese_errors(self, texts):
        # https://huggingface.co/bmd1905/vietnamese-correction-v2
        predictions = self.corrector(texts, max_length=self.max_correction_length, truncation=True)
        return [prediction['generated_text'] for prediction in predictions]


    def word_segment(self, text):
        if self.word_segmenter:
            words = self.word_segmenter.tokenize(text)
            return ' '.join(sum(words, [])) # Flatten the list of words
        print('There is no VnCoreNLP word segmenter loaded. Please check the VnCoreNLP jar file.')
        return text


    def process_text(self, text, normalize_tone=True, segment=True):
        text = text.lower()
        if normalize_tone:
            text = VietnameseToneNormalizer.normalize_unicode(text)
            text = VietnameseToneNormalizer.normalize_sentence_typing(text)
        text = VietnameseTextCleaner.process_text(text)
        text = self.normalize_teencodes(text)
        return self.word_segment(text) if segment else text


    def process_batch(self, texts, correct_errors=True):
        if correct_errors:
            texts = [self.process_text(text, normalize_tone=True, segment=False) for text in texts]
            texts = self.correct_vietnamese_errors(texts)
            return [self.process_text(text, normalize_tone=False, segment=True) for text in texts]
        return [self.process_text(text, normalize_tone=True, segment=True) for text in texts]


    def close_vncorenlp(self):
        if self.word_segmenter:
            print('Closing VnCoreNLP word segmenter...')
            self.word_segmenter.close()


if __name__ == '__main__':
    # You should be careful when using single word replacement for teencodes, because it can cause misinterpretation.
    # For example, 'giá': ['price', 'gia'] can replace the word 'gia' in 'gia đình', making it become 'giá đình'.
    extra_teencodes = {
        'khách sạn': ['ks'], 'nhà hàng': ['nhahang'], 'nhân viên': ['nv'],
        'cửa hàng': ['store', 'sop', 'shopE', 'shop'],
        'sản phẩm': ['sp', 'product'], 'hàng': ['hàg'],
        'giao hàng': ['ship', 'delivery', 'síp'], 'đặt hàng': ['order'],
        'chuẩn chính hãng': ['authentic', 'aut', 'auth'], 'hạn sử dụng': ['date', 'hsd'],
        'điện thoại': ['dt'],  'facebook': ['fb', 'face'],
        'nhắn tin': ['nt', 'ib'], 'trả lời': ['tl', 'trl', 'rep'],
        'feedback': ['fback', 'fedback'], 'sử dụng': ['sd'], 'xài': ['sài'],
    }

    preprocessor = VietnameseTextPreprocessor(vncorenlp_dir='./VnCoreNLP', extra_teencodes=extra_teencodes, max_correction_length=512)
    sample_texts = [
        'Ga giường không sạch, nhân viên quên dọn phòng một ngày. Chất lựơng "ko" đc thỏai mái 😔',
        'Cám ơn Chudu24 rất nhiềuGia đình tôi có 1 kỳ nghỉ vui vẻ.Resort Bình Minh nằm ở vị trí rất đẹp, theo đúng tiêu chuẩn, còn về ăn sáng thì wa dở, chỉ có 2,3 món để chọn',
        'Giá cả hợp líĂn uống thoả thíchGiữ xe miễn phíKhông gian bờ kè thoáng mát Có phòng máy lạnhMỗi tội lúc quán đông thì đợi hơi lâu',
        'May lần trước ăn mì k hà, hôm nay ăn thử bún bắp bò. Có chả tôm viên ăn lạ lạ. Tôm thì k nhiều, nhưng vẫn có tôm thật ở nhân bên trong. ',
        'Ngồi ăn Cơm nhà *tiền thân là quán Bão* Phần vậy là 59k nha. Trưa từ 10h-14h, chiều từ 16h-19h. À,có sữa hạt sen ngon lắmm. #food #foodpic #foodporn #foodholic #yummy #deliciuous'
    ]
    preprocessed_texts = preprocessor.process_batch(sample_texts, correct_errors=True)
    preprocessor.close_vncorenlp()
    print(preprocessed_texts)

VnCoreNLP word segmenter is loaded successfully.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/961 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Device set to use cuda:0


bmd1905/vietnamese-correction-v2 is loaded successfully.


Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Closing VnCoreNLP word segmenter...
['ga giường không sạch nhân_viên quên dọn phòng một ngày chất_lượng không được thoải_mái', 'cám_ơn chudu24 rất nhiều gia_đình tôi có 1 kỳ nghỉ vui_vẻ resort bình_minh nằm ở vị_trí rất đẹp theo đúng tiêu_chuẩn còn về ăn sáng thì quá dở chỉ có 2 3 món để chọn', 'giá_cả hợp_lý uống thích giữ xe miễn_phí không_gian bờ kè thoáng mát có phòng máy_lạnh mỗi_tội lúc quán đông thì đợi hơi lâu', 'mấy lần trước ăn mì không hà hôm_nay ăn thử bún bắp bò có chả tôm viên ăn lạ_lạ tôm thì không nhiều nhưng vẫn có tôm thật ở nhân bên trong', 'ngồi ăn cơm nhà tiền_thân là quán bão phần vậy là 59k nha trưa từ 10h 14h chiều từ 16h 19h à có sữa hạt sen ngon lắm']


In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

def preprocess_restaurant_dataset():
    """
    Tiền xử lý dữ liệu cho 3 tập train, dev, test của Restaurant dataset
    """

    # Đường dẫn file dữ liệu gốc
    restaurant_train_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Restaurant_Final/1-VLSP2018-SA-Restaurant-train.csv'
    restaurant_val_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Restaurant_Final/2-VLSP2018-SA-Restaurant-dev.csv'
    restaurant_test_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Restaurant_Final/3-VLSP2018-SA-Restaurant-test.csv'

    # Đường dẫn file output đã tiền xử lý
    output_train_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Final_nlp/final_nlp_processed_train.csv'
    output_val_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Final_nlp/final_nlp_processed_dev.csv'
    output_test_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Final_nlp/final_nlp_processed_test.csv'

    # Tạo thư mục output nếu chưa tồn tại
    os.makedirs(os.path.dirname(output_train_path), exist_ok=True)

    # Khởi tạo preprocessor với các teencode mở rộng cho domain nhà hàng
    extra_teencodes = {
        'khách sạn': ['ks'], 'nhà hàng': ['nhahang'], 'nhân viên': ['nv'],
        'cửa hàng': ['store', 'sop', 'shopE', 'shop'],
        'sản phẩm': ['sp', 'product'], 'hàng': ['hàg'],
        'giao hàng': ['ship', 'delivery', 'síp'], 'đặt hàng': ['order'],
        'chuẩn chính hãng': ['authentic', 'aut', 'auth'], 'hạn sử dụng': ['date', 'hsd'],
        'điện thoại': ['dt'],  'facebook': ['fb', 'face'],
        'nhắn tin': ['nt', 'ib'], 'trả lời': ['tl', 'trl', 'rep'],
        'feedback': ['fback', 'fedback'], 'sử dụng': ['sd'], 'xài': ['sài'],
    }

    print("Đang khởi tạo Vietnamese Text Preprocessor...")
    preprocessor = VietnameseTextPreprocessor(
        vncorenlp_dir='./VnCoreNLP',
        extra_teencodes=extra_teencodes,
        max_correction_length=512
    )

    # Danh sách các file cần xử lý
    files_to_process = [
        (restaurant_train_path, output_train_path, 'Train'),
        (restaurant_val_path, output_val_path, 'Dev'),
        (restaurant_test_path, output_test_path, 'Test')
    ]

    for input_path, output_path, dataset_name in files_to_process:
        print(f"\n=== Đang xử lý {dataset_name} dataset ===")

        try:
            # Đọc file CSV
            print(f"Đọc file: {input_path}")
            df = pd.read_csv(input_path)

            print(f"Số lượng mẫu: {len(df)}")
            print(f"Các cột trong dataset: {list(df.columns)}")

            # Kiểm tra cột text (có thể là 'text', 'comment', 'review', etc.)
            text_column = None
            possible_text_columns = ['text', 'comment', 'review', 'content', 'sentence']

            for col in possible_text_columns:
                if col in df.columns:
                    text_column = col
                    break

            if text_column is None:
                print(f"Không tìm thấy cột text phù hợp. Các cột hiện tại: {list(df.columns)}")
                # Giả sử cột đầu tiên là text nếu không tìm thấy
                text_column = df.columns[0]
                print(f"Sử dụng cột đầu tiên làm text column: {text_column}")

            # Lấy danh sách text cần xử lý
            texts = df[text_column].fillna("").astype(str).tolist()

            print(f"Đang tiền xử lý {len(texts)} văn bản...")

            # Chia nhỏ batch để xử lý (tránh quá tải memory)
            batch_size = 32
            processed_texts = []

            for i in tqdm(range(0, len(texts), batch_size), desc=f"Processing {dataset_name}"):
                batch = texts[i:i+batch_size]
                # Xử lý batch với correction errors
                processed_batch = preprocessor.process_batch(batch, correct_errors=True)
                processed_texts.extend(processed_batch)

            # Cập nhật DataFrame với text đã xử lý
            df[text_column + '_processed'] = processed_texts

            # Lưu file đã xử lý
            df.to_csv(output_path, index=False, encoding='utf-8')
            print(f"Đã lưu file đã xử lý: {output_path}")

            # Hiển thị một vài ví dụ
            print("\n--- Ví dụ kết quả xử lý ---")
            for i in range(min(3, len(df))):
                print(f"Gốc: {texts[i][:100]}...")
                print(f"Đã xử lý: {processed_texts[i][:100]}...")
                print("-" * 50)

        except Exception as e:
            print(f"Lỗi khi xử lý {dataset_name}: {str(e)}")
            continue

    # Đóng VnCoreNLP
    preprocessor.close_vncorenlp()
    print("\n=== Hoàn thành tiền xử lý tất cả dataset ===")


# Chạy tiền xử lý
if __name__ == '__main__':
    preprocess_restaurant_dataset()


Đang khởi tạo Vietnamese Text Preprocessor...
VnCoreNLP word segmenter is loaded successfully.


Device set to use cuda:0


bmd1905/vietnamese-correction-v2 is loaded successfully.

=== Đang xử lý Train dataset ===
Đọc file: /content/drive/MyDrive/Colab Notebooks/Dataset/Restaurant_Final/1-VLSP2018-SA-Restaurant-train.csv
Số lượng mẫu: 2961
Các cột trong dataset: ['Review', 'AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']
Không tìm thấy cột text phù hợp. Các cột hiện tại: ['Review', 'AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']
Sử dụng cột đầu tiên làm text column: Review
Đang tiền xử lý 2961 văn bản...


Processing Train:   0%|          | 0/93 [00:00<?, ?it/s]Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/

Đã lưu file đã xử lý: /content/drive/MyDrive/Colab Notebooks/Dataset/Final_nlp/final_nlp_processed_train.csv

--- Ví dụ kết quả xử lý ---
Gốc: _ Ảnh chụp từ hôm qua, đi chơi với gia đình và 1 nhà họ hàng đang sống tại Sài Gòn. _ Hôm qua đi ăn ...
Đã xử lý: ảnh chụp từ hôm_qua đi chơi với gia_đình và 1 nhà họ hàg đang sống tại xài gòn hôm_qua đi ăn trưa mu...
--------------------------------------------------
Gốc: _Hương vị thơm ngon, ăn cay cay rất thích, nêm nếm vừa miệng. Ngoài ra menu quán cũng nhiều món khác...
Đã xử lý: hương_vị thơm ngon_ăn cay cay rất thích nêm nếm vừa_miệng ngoài_ra menu quán cũng nhiều món khác nha...
--------------------------------------------------
Gốc: - 1 bàn tiệc hoành tráng 3 đứa ăn no muốn tắt thở mà giá chỉ 228k (ăn trung đợt giảm 10%), mình thíc...
Đã xử lý: 1 bàn tiệc hoành_tráng 3 đứa ăn no muốn tắt_thở mà giá chỉ 228k ăn trung đợt giảm 10k mình thích nhấ...
--------------------------------------------------

=== Đang xử lý Dev dataset ===
Đọc file

Processing Dev:   0%|          | 0/41 [00:00<?, ?it/s]Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/do

Đã lưu file đã xử lý: /content/drive/MyDrive/Colab Notebooks/Dataset/Final_nlp/final_nlp_processed_dev.csv

--- Ví dụ kết quả xử lý ---
Gốc: - 60k/1con- Tu hài to, siêu béo siêu ngon, nướng mỡ hành thơm phức, béo ngậy- Đĩa tu hài đem ra nóng...
Đã xử lý: 60k1 con tu hài to siêu béo siêu ngon nướng mỡ hành thơm_phức béo ngậy đĩa tu hài đem ra nóng_hổi gắ...
--------------------------------------------------
Gốc: - Ăn hai người no căng :3- Nhiều đồ ăn kèm như phồng tôm, bánh mỳ,kim chi, khoai tây chiên🍟🍻- Hôm qu...
Đã xử lý: ăn hai người no căng 3 nhiều đồ_ăn kèm như phồng_tôm bánh mỳ kim chi khoai_tây chiên hôm_qua mình gọ...
--------------------------------------------------
Gốc: - Ấn tượng đầu là quán siêu to siêu thoáng lại còn bonus thêm quả máy hút mùi ở mỗi bàn. Quá hiện đạ...
Đã xử lý: ấn_tượng đầu là quán siêu to siêu thoáng lại còn bonus thêm quả máy hút mùi ở mỗi bàn quá hiện_đại l...
--------------------------------------------------

=== Đang xử lý Test dataset ===
Đọc file:

Processing Test:   0%|          | 0/16 [00:00<?, ?it/s]Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/d

Đã lưu file đã xử lý: /content/drive/MyDrive/Colab Notebooks/Dataset/Final_nlp/final_nlp_processed_test.csv

--- Ví dụ kết quả xử lý ---
Gốc: Đây là 1 trong những quán mà mình thích vì vị trà đậm và thơm cũng như mùi vị đặc trưng hơn hẳn nhữn...
Đã xử lý: đây là 1 trong những quán mà mình thích vì vị trà đậm và thơm cũng như mùi_vị đặc_trưng hơn hẳn nhữn...
--------------------------------------------------
Gốc: Bữa lướt facebook nhỏ bạn thấy có check-in ly trà sữa khổng lồ cùng viên phô mai to ú nụ. Mình bị hấ...
Đã xử lý: bữa lướt facebook nhỏ bạn thấy có check in ly trà sữa khổng_lồ cùng viên phô_mai to ú nụ mình bị hấp...
--------------------------------------------------
Gốc: Nghe phông phanh về trà sữa có viên phô mai to ú ù lâu rồi mà chưa có dịp đi thử. Thế là hôm qua cũn...
Đã xử lý: nghe phong_phanh về trà sữa có viên phô_mai to ú ù lâu rồi mà chưa có dịp đi thử thế_là hôm_qua cũng...
--------------------------------------------------
Closing VnCoreNLP word segmenter...

=== 